# Basic Usage

This example on CO illustates all the features of using horton-part package.

The format checkpoint file wavefunction is included in "docs/notebooks/data/co.fchk" and is
read using the [IOData](https://github.com/theochem/iodata) package.

In [ ]:
# Import required libraries.
import numpy as np
from gbasis.evals.eval import evaluate_basis
from gbasis.wrappers import from_iodata
from grid import BeckeWeights, ExpRTransform, MolGrid, UniformInteger
from iodata import load_one

np.set_printoptions(precision=3, linewidth=np.inf, suppress=True)

In [ ]:
def prepare_grid_and_dens():
    """Prepare molecular grid and density."""
    mol = load_one("data/co.fchk")

    # Specify the integration grid
    rtf = ExpRTransform(5e-4, 2e1, 120 - 1)
    uniform_grid = UniformInteger(120)
    rgrid = rtf.transform_1d_grid(uniform_grid)
    becke = BeckeWeights()
    grid = MolGrid.from_preset(
        mol.atnums, mol.atcoords, rgrid, "fine", becke, rotate=False, store=True
    )

    # Get the spin-summed density matrix
    one_rdm = mol.one_rdms.get("post_scf", mol.one_rdms.get("scf"))
    basis, coord_types = from_iodata(mol)
    basis_grid = evaluate_basis(basis, grid.points, coord_type=coord_types)
    rho = np.einsum("ab,bp,ap->p", one_rdm, basis_grid, basis_grid, optimize=True)
    nelec = grid.integrate(rho)
    print(f"The number of electrons: {nelec}")
    print(f"Coordinates of the atoms \n {mol.atcoords}")
    print(f"Atomic numbers of the atom \n {mol.atnums}")
    return mol, grid, rho


def prepare_argument_dict():
    """Prepare basic input arguments for all AIM methods."""
    mol, grid, rho = prepare_grid_and_dens()
    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
    }
    return kwargs


def print_results(part):
    """Print partitioning results."""
    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

## ISA method

In [ ]:
from horton_part import ISAWPart


def main_isa():
    """The main entry to run ISA method"""
    part = ISAWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_isa()

## MBIS method

In [ ]:
from horton_part import MBISWPart


def main_mbis():
    """The main entry to run MBIS method"""
    part = MBISWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_mbis()

## GISA method

In [ ]:
from horton_part import GaussianISAWPart


def main_gisa():
    kwargs = prepare_argument_dict()
    kwargs["solver"] = 1
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_gisa()

## Local LISA method

### Non-linear optimization problem

#### Convex optimization method


In [ ]:
from horton_part import LisaConvexOptWPart


def main_lisa():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict()
    part = LisaConvexOptWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa()

#### Trust-region method

In [ ]:
from horton_part import LisaTrustConstraintImpWPart


def main_lisa_301():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict()
    part = LisaTrustConstraintImpWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_301()

One can also add constraint explicitly

In [ ]:
from horton_part import LisaTrustConstraintExpWPart


def main_lisa_302():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict()
    part = LisaTrustConstraintExpWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_302()

### Non-linear equations (fixed-point equations)

#### Self-consistent method


In [ ]:
from horton_part import LisaSelfConsistentWPart


def main_lisa_201():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict()
    part = LisaSelfConsistentWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_201()

#### DIIS

In [ ]:
from horton_part import LisaDIISWPart


def main_lisa_202():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict()
    part = LisaDIISWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_202()

#### Newton method

In [ ]:
from horton_part import LisaNewtonWPart


def main_lisa_203():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict()
    part = LisaNewtonWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_203()

## Global LISA 

### Convex optimization problem

In [ ]:
from horton_part.lisa_g import GLisaConvexOptWPart


def main_lisa_g_101():
    """Global LISA by solving convex optimization problem."""
    part = GLisaConvexOptWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_lisa_g_101()

### Convex optimization problem (with negative parameters allowed)


In [ ]:
from horton_part.lisa_g import GLisaConvexOptNWPart


def main_lisa_g_104():
    """Global LISA by solving constraint optimization problem (using trust constraint solver."""
    part = GLisaConvexOptNWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_lisa_g_104()

### Constraint optimization problem 

The optimization problem is solved by using trust constraint solver and all parameters are non-negative.


In [ ]:
from horton_part.lisa_g import GLisaTrustConstrainWPart


def main_lisa_g_301():
    """Global LISA by solving constraint optimization problem (using trust constraint solver."""
    part = GLisaTrustConstrainWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_lisa_g_301()

### Constraint optimization problem (with negative parameters allowed)

The optimization problem is solved by using trust constraint solver and negative parameter is allowed.


In [ ]:
from horton_part.lisa_g import GLisaTrustConstrainNWPart


def main_lisa_g_302():
    """Global LISA by solving constraint optimization problem (using trust constraint solver with negative parameters allowed.)"""
    part = GLisaTrustConstrainNWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_lisa_g_302()

### Fixed-point problem by self-consistent solver

An equivalent fixed-point problem is addressed by using self-consistent solver and non-negative parameters are guaranteed.


In [ ]:
from horton_part.lisa_g import GLisaSelfConsistentWPart


def main_lisa_g_201():
    """Global LISA with self-consistent solver."""
    part = GLisaSelfConsistentWPart(**prepare_argument_dict())
    part.do_all()
    print_results(part)


main_lisa_g_201()

### Fixed-point problem by DIIS solver

An equivalent fixed-point problem is addressed by using self-consistent solver and non-negative parameters are guaranteed.


In [ ]:
from horton_part.lisa_g import GLisaDIISWPart


def main_lisa_g_206():
    """Global LISA with DIIS solver."""
    part = GLisaDIISWPart(**prepare_argument_dict())
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])


main_lisa_g_206()